In [1]:
%pwd

'd:\\Work\\Text-Summarization-NLP\\research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'd:\\Work\\Text-Summarization-NLP'

## Creating entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path 
   unzip_dir: Path

In [5]:
## Create configuration manager

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

## Create components

In [8]:
import os
import urllib.request as requests
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name , headers = requests.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
                )
            logger.info(f"{file_name} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

## Create pipeline

In [10]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-23 17:38:51,237: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-23 17:38:51,239: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-23 17:38:51,241: INFO: common: created directory at: artifacts]
[2023-11-23 17:38:51,242: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-23 17:38:55,180: INFO: 42848312: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
content-disposition: attachment
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "c1b6d464322975cff5b61efcec056163f6fbc488dd31283f5624d3c4501a2d27"
Last-Modified: Thu, 23 Nov 2023 12:08:51 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4162:ACB28:12663F:1EE51D:655F40D2
Accept-Ranges: bytes